In [2]:
import os
import pandas as pd
import matplotlib as plt

In [3]:
df = pd.read_csv('total_run_1_parsed_outputs.csv')
print(df)

      INDEX        TRIAL          PROMPT                    MODEL  \
0         0  total_run_1  prompt_prompt1             gemma2-9b-it   
1         1  total_run_1  prompt_prompt1             gemma2-9b-it   
2         2  total_run_1  prompt_prompt1             gemma2-9b-it   
3         3  total_run_1  prompt_prompt1             gemma2-9b-it   
4         4  total_run_1  prompt_prompt1             gemma2-9b-it   
...     ...          ...             ...                      ...   
2368   2368  total_run_1  prompt_prompt1  llama-3.1-70b-versatile   
2369   2369  total_run_1  prompt_prompt1  llama-3.1-70b-versatile   
2370   2370  total_run_1  prompt_prompt1  llama-3.1-70b-versatile   
2371   2371  total_run_1  prompt_prompt1  llama-3.1-70b-versatile   
2372   2372  total_run_1  prompt_prompt1  llama-3.1-70b-versatile   

           SOURCE DATA_ID            FILENAME  VULN_MODEL  VULN_TRUTH  
0     cwe_samples    1004   1004_Bad.java.txt        True        True  
1     cwe_samples    1004  

In [4]:
countCorrect = ((df['VULN_MODEL']) == (df['VULN_TRUTH'])).sum()
print(f'Correct: {countCorrect}')
countFalse = ((df['VULN_MODEL']) != (df['VULN_TRUTH'])).sum()
print(f'Mismatched: {countFalse}')

Correct: 1931
Mismatched: 442


In [7]:
df_llama = df[df["MODEL"] == "llama-3.1-70b-versatile" ]
countCorrect = ((df_llama['VULN_MODEL']) == (df_llama['VULN_TRUTH'])).sum()
print(f'Correct: {countCorrect}')
countFalse = ((df_llama['VULN_MODEL']) != (df_llama['VULN_TRUTH'])).sum()
print(f'Mismatched: {countFalse}')
print(f'Total: {len(df_llama)}')
print(f"Score: {countCorrect/len(df_llama)*100}%")

Correct: 969
Mismatched: 220
Total: 1189
Score: 81.49705634987384%


In [8]:
df_gemma = df[df["MODEL"] == "gemma2-9b-it" ]
countCorrect = ((df_gemma['VULN_MODEL']) == (df_gemma['VULN_TRUTH'])).sum()
print(f'Correct: {countCorrect}')
countFalse = ((df_gemma['VULN_MODEL']) != (df_gemma['VULN_TRUTH'])).sum()
print(f'Mismatched: {countFalse}')
print(f'Total: {len(df_gemma)}')
print(f"Score: {countCorrect/len(df_gemma)*100}%")

Correct: 962
Mismatched: 222
Total: 1184
Score: 81.25%


In [12]:
# Group by the relevant columns
grouped = df.groupby(['PROMPT', 'SOURCE', 'DATA_ID', 'FILENAME'])

# Create a list to hold the final rows
final_rows = []

# Iterate over each group to create the consolidated rows
for name, group in grouped:
    new_row = {
        'PROMPT': name[0],
        'SOURCE': name[1],
        'DATA_ID': name[2],
        'FILENAME': name[3],
        'VULN_TRUTH': group['VULN_TRUTH'].iloc[0]
    }
    
    # Add each model's VULN_MODEL value to the row
    for _, row in group.iterrows():
        new_row[row['MODEL']] = row['VULN_MODEL']
    
    final_rows.append(new_row)

# Convert the list of rows into a new DataFrame
final_df = pd.DataFrame(final_rows)

# Display the final DataFrame
print(final_df)

              PROMPT           SOURCE  \
0     prompt_prompt1  cpython_samples   
1     prompt_prompt1  cpython_samples   
2     prompt_prompt1  cpython_samples   
3     prompt_prompt1  cpython_samples   
4     prompt_prompt1      cwe_samples   
...              ...              ...   
1184  prompt_prompt1      cwe_samples   
1185  prompt_prompt1      cwe_samples   
1186  prompt_prompt1      cwe_samples   
1187  prompt_prompt1      cwe_samples   
1188  prompt_prompt1      cwe_samples   

                                       DATA_ID                FILENAME  \
0     00464bbed66e5f64bdad7f930b315a88d5afccae        ssl_after.py.txt   
1     00464bbed66e5f64bdad7f930b315a88d5afccae       ssl_before.py.txt   
2     00464bbed66e5f64bdad7f930b315a88d5afccae   test_ssl_after.py.txt   
3     00464bbed66e5f64bdad7f930b315a88d5afccae  test_ssl_before.py.txt   
4                                         1004       1004_Bad.java.txt   
...                                        ...                 

In [11]:
# Filter rows where any of the VULN_MODEL values don't match
model_columns = final_df.columns.difference(['PROMPT', 'SOURCE', 'DATA_ID', 'FILENAME', 'VULN_TRUTH'])

# Filtering condition: count unique values in non-NaN entries, include row if more than 1 unique value
filtered_df = final_df[final_df[model_columns].apply(lambda row: row.dropna().nunique() > 1, axis=1)]

# Display the filtered DataFrame
print("\nFiltered DataFrame where VULN_MODEL values don't match:")
print(filtered_df)


Filtered DataFrame where VULN_MODEL values don't match:
              PROMPT       SOURCE DATA_ID             FILENAME  VULN_TRUTH  \
5     prompt_prompt1  cwe_samples    1004   1004_Good.java.txt       False   
19    prompt_prompt1  cwe_samples    1041   1041_Good.java.txt       False   
32    prompt_prompt1  cwe_samples    1071  1071_Bad_1.java.txt        True   
38    prompt_prompt1  cwe_samples     110     110_Bad.java.txt        True   
46    prompt_prompt1  cwe_samples     112   112_Bad_1.java.txt        True   
...              ...          ...     ...                  ...         ...   
1123  prompt_prompt1  cwe_samples     862      862_Bad.php.txt        True   
1136  prompt_prompt1  cwe_samples     908     908_Bad.java.txt        True   
1139  prompt_prompt1  cwe_samples     908       908_Good.c.txt       False   
1145  prompt_prompt1  cwe_samples     909       909_Good.c.txt       False   
1148  prompt_prompt1  cwe_samples     915       915_Bad.js.txt        True   

      